In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

### Syntax

In [2]:
data Register = ACC 
              | NIL
           -- | ANY
           -- | LAST
              | IN  
              | OUT
              deriving (Show, Read)

data Operand = Reg Register
             | Con Int
             deriving (Show, Read)

type Location = Int           

data Operation = NOP
               | MOV Operand Register
               | SWP
               | SAV
               | ADD Operand
               | SUB Operand
               | NEG
               | JMP Location
               | JEZ Location
               | JNZ Location
               | JGZ Location
               | JLZ Location
            -- | JRO Operand
               deriving (Show, Read)

type Node = [Operation]

### Parsing

In [3]:
import Text.Read
import Data.Maybe
import Control.Monad

import Parsing

In [4]:
word = token $ many upper

register :: Parser Register
register = do xs <- word
              guard $ isJust (readMaybe xs :: Maybe Register)
              return (read xs :: Register)

operand :: Parser Operand
operand =  Reg <$> register
       <|> Con <$> integer

operation :: Parser Operation
operation =  do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operation)
                return (read xs :: Operation)
         <|> do symbol "MOV"
                o <- operand
                MOV o <$> register
         <|> do w <- word
                guard $ elem w ["ADD","SUB"]
                o <- operand
                return (read (w ++ " (" ++ show o ++ ")") :: Operation) -- messy
         <|> do w <- word
                guard $ elem w ["JMP","JEZ","JNZ","JGZ","JLZ"]
                i <- integer
                return (read (w ++ " (" ++ show i ++ ")") :: Operation)
                
node = many operation

### Testing

In [5]:
exampleNode = unlines [
              "MOV IN  ACC",
              "SWP        ",
              "MOV 1   ACC",
              "           ",
              "SWP        ", -- line 3
              "JEZ 9      ",
              "SUB 1      ",
              "SWP        ",
              "ADD ACC    ",
              "JMP 3      ",
              "           ",
              "SWP        ", -- line 9
              "MOV ACC OUT"]

putStr exampleNode

MOV IN  ACC
SWP        
MOV 1   ACC
           
SWP        
JEZ 9      
SUB 1      
SWP        
ADD ACC    
JMP 3      
           
SWP        
MOV ACC OUT

In [6]:
parse node exampleNode

[([MOV (Reg IN) ACC,SWP,MOV (Con 1) ACC,SWP,JEZ 9,SUB (Con 1),SWP,ADD (Reg ACC),JMP 3,SWP,MOV (Reg ACC) OUT],"")]